# Imports

In [3]:
import numpy as np
import pandas as pd

from darts import TimeSeries
from darts.models import TransformerModel
from darts.metrics import mae

from powderalert.ml_logic.data import fetch_weather_data
from powderalert.ml_logic.preprocessor import preprocess

import optuna
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# Improving Model Performance | Target = Snowfall

## Getting Started | Import basic dataset & preprocess

In [ ]:
# for future reference - get data directly from api?
# fetch_weather_data()

In [4]:
# in the meantime use preprocessed csv (refer to notebook_Anita-Gei_get_preprocessed_data.ipynb for more info)

df = pd.read_csv('/home/anita/code/MadMax1995bb/powder_alert2.0/raw_data/openmeteo_api_NEW_train_dataset.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.drop(columns='Unnamed: 0')
df = df.set_index('date')
df.tail(2)

,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,...,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,sunshine_duration
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,-13.721,80.381676,-16.371,0.0,0.0,0.0,1.23,3.0,1016.9,804.64825,...,2.0,100.0,0.0,0.041773,6.763786,12.429127,154.79890,190.00792,26.28,0.679
2024-01-01 23:00:00,-13.421,78.443700,-16.371,0.0,0.0,0.0,1.23,1.0,1015.7,803.91144,...,0.0,40.0,0.0,0.047036,6.439876,12.605142,153.43501,181.63654,27.00,0.679


In [5]:
print(df.shape)

(131496, 22)


## Feature Engineering

In [6]:
#create categorical feature winter
df['month'] = df.index.month
df["is_winter"] = df['month'].apply(lambda x: 1 if x in [12, 1, 2] else 0)
df = df.drop(columns='month')

In [7]:
# Create lag features for snowfall and precipitation
lags = [1, 3, 6, 12]
for lag in lags:
    df[f'snowfall_lag_{lag}'] = df['snowfall'].shift(lag)
    df[f'precipitation_lag_{lag}'] = df['precipitation'].shift(lag)

df.tail(2)

,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,...,sunshine_duration,is_winter,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,-13.721,80.381676,-16.371,0.0,0.0,0.0,1.23,3.0,1016.9,804.64825,...,0.679,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-01 23:00:00,-13.421,78.443700,-16.371,0.0,0.0,0.0,1.23,1.0,1015.7,803.91144,...,0.679,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Add rolling statistics
df['cloud_cover_rolling_mean_3h'] = df['cloud_cover'].rolling(window=3).mean()
df['precipitation_rolling_std_6h'] = df['precipitation'].rolling(window=6).std()
df.tail(2)

,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,...,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,-13.721,80.381676,-16.371,0.0,0.0,0.0,1.23,3.0,1016.9,804.64825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.333333,0.0
2024-01-01 23:00:00,-13.421,78.443700,-16.371,0.0,0.0,0.0,1.23,1.0,1015.7,803.91144,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.000000,0.0


In [9]:
# Drop rows with NaN values (introduced by lags and rolling)
df = df.dropna()
print(df.shape)

(131484, 33)


In [10]:
df.tail()

,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,...,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 19:00:00,-13.821,86.608580,-15.571000,0.0,0.0,0.0,1.23,3.0,1017.1,804.73530,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.333333,0.0
2024-01-01 20:00:00,-14.571,86.166550,-16.371000,0.0,0.0,0.0,1.23,1.0,1017.4,804.43820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.666667,0.0
2024-01-01 21:00:00,-14.571,84.035320,-16.671001,0.0,0.0,0.0,1.23,3.0,1017.1,804.20090,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.666667,0.0
2024-01-01 22:00:00,-13.721,80.381676,-16.371000,0.0,0.0,0.0,1.23,3.0,1016.9,804.64825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.333333,0.0
2024-01-01 23:00:00,-13.421,78.443700,-16.371000,0.0,0.0,0.0,1.23,1.0,1015.7,803.91144,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.000000,0.0


# Preprocess DataFrame

In [11]:
df = df.drop_duplicates()

y = df[['snowfall']]
X = df.drop(columns="snowfall")

In [18]:
y = y.reset_index()
y.tail(2)

,date,snowfall
131482,2024-01-01 22:00:00,0.0
131483,2024-01-01 23:00:00,0.0


In [13]:
X.tail(2)

,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,...,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,-13.721,80.381676,-16.371,0.0,0.0,1.23,3.0,1016.9,804.64825,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.333333,0.0
2024-01-01 23:00:00,-13.421,78.443700,-16.371,0.0,0.0,1.23,1.0,1015.7,803.91144,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.000000,0.0


In [16]:
X_preprocessed = preprocess(X)
X_preprocessed.tail(2)

✅ Processed data, with shape (131484, 38)


,weather_code_encoded,is_winter_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,pressure_msl,surface_pressure,...,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
131482,3.0,1.0,-1.911528,0.072457,-1.882966,-0.38975,-0.272951,1.403606,-0.129525,-1.404396,...,-0.289649,-0.389783,0.191301,-0.471575,-0.707025,1.224756,0.00051,1.414096,0.004523,1.419886
131483,1.0,1.0,-1.875463,-0.040453,-1.882966,-0.38975,-0.272951,1.403606,-0.280589,-1.495112,...,-0.289649,-0.389783,0.323438,-0.471575,-0.365944,1.366036,0.00051,1.414096,0.004523,1.419886


In [19]:
df = y.join(X_preprocessed)
print(df.shape)
df.tail(2)

(131484, 40)


,date,snowfall,weather_code_encoded,is_winter_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,...,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
131482,2024-01-01 22:00:00,0.0,3.0,1.0,-1.911528,0.072457,-1.882966,-0.38975,-0.272951,1.403606,...,-0.289649,-0.389783,0.191301,-0.471575,-0.707025,1.224756,0.00051,1.414096,0.004523,1.419886
131483,2024-01-01 23:00:00,0.0,1.0,1.0,-1.875463,-0.040453,-1.882966,-0.38975,-0.272951,1.403606,...,-0.289649,-0.389783,0.323438,-0.471575,-0.365944,1.366036,0.00051,1.414096,0.004523,1.419886


In [20]:
# save as csv
import os

current_dir = os.getcwd()
relative_path = os.path.dirname(current_dir)
file_path = os.path.join(relative_path, "raw_data")

df.to_csv(os.path.join(file_path,'adj_historical_data_preprocessed.csv'))

## Use Subset of Data while finding the "best" Model!

In [28]:
train_df = df[-30000:]
print(train_df.shape)
train_df

(30000, 38)


,snowfall,weather_code_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,pressure_msl,surface_pressure,...,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h
date,,,,,,,,,,,,,,,,,,,,,
2020-07-31 00:00:00,0.0,0.0,1.292371,-0.173876,1.301722,-0.389763,-0.272938,-0.905450,0.449452,1.193718,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,-1.460140,0.0
2020-07-31 01:00:00,0.0,1.0,1.256307,-0.221554,1.244856,-0.389763,-0.272938,-0.905450,0.399097,1.131659,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,-1.396946,0.0
2020-07-31 02:00:00,0.0,1.0,1.112051,-0.096769,1.150080,-0.389763,-0.272938,-0.905450,0.373920,1.022414,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,-1.225418,0.0
2020-07-31 03:00:00,0.0,0.0,1.106041,-0.127916,1.131125,-0.389763,-0.272938,-0.905450,0.386509,1.028612,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,-1.216390,0.0
2020-07-31 04:00:00,0.0,0.0,1.124073,-0.172170,1.131125,-0.389763,-0.272938,-0.905450,0.399097,1.049746,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,-1.550417,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 19:00:00,0.0,3.0,-1.923330,0.435258,-1.781667,-0.389763,-0.272938,1.403548,-0.104446,-1.393728,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.805826,0.0
2024-01-01 20:00:00,0.0,1.0,-2.013490,0.409503,-1.882762,-0.389763,-0.272938,1.403548,-0.066680,-1.430309,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.164856,0.0
2024-01-01 21:00:00,0.0,3.0,-2.013490,0.285328,-1.920672,-0.389763,-0.272938,1.403548,-0.104446,-1.459527,...,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.137773,0.0


## Create Time Series

In [25]:
feature_columns = train_df.drop(columns=['snowfall']).columns.tolist()

snowfall_series = TimeSeries.from_dataframe(train_df, value_cols='snowfall')
feature_series = TimeSeries.from_dataframe(train_df, value_cols=feature_columns)

## Split into Train, Validation and Test Sets

In [26]:
y_train_val, y_test = snowfall_series.split_before(0.8)
y_train, y_val = y_train_val.split_before(0.8)

X_train_val, X_test = feature_series.split_before(0.8)
X_train, X_val = X_train_val.split_before(0.8)

## Optimize Hyperparamters with Optuna

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [21]:
#use only with gpu!
def objective(trial):
    d_model = trial.suggest_categorical("d_model", [32, 64, 128])
    nhead = trial.suggest_categorical("nhead", [2, 4, 8])
    dropout = trial.suggest_float("dropout", 0.01, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    num_encoder_layers = trial.suggest_int("num_encoder_layers", 1, 3)
    num_decoder_layers = trial.suggest_int("num_decoder_layers", 1, 3)
    dim_feedforward = trial.suggest_categorical("dim_feedforward", [128, 256, 512])
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)

    es = EarlyStopping(
      monitor="val_loss",
      patience=5,
      min_delta=0.05,
      mode='min',
    )

    model = TransformerModel(
        input_chunk_length=48,
        output_chunk_length=48,
        batch_size=batch_size,
        n_epochs=100,
        model_name="transformer_snowfall_optuna",
        d_model=d_model,
        nhead=nhead,
        dim_feedforward=dim_feedforward,
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        dropout=dropout,
        optimizer_kwargs={"lr": lr},
        pl_trainer_kwargs={
            "accelerator": "gpu",
            "devices": -1,
            "callbacks": [es]},
        random_state=42
    )

    for step in range(100):
        model.fit(
          series=y_train,
          past_covariates=X_train,
          val_series=y_val,
          val_past_covariates=X_val,
          verbose=False
        )

        intermediate_value = mae(y_test[:48], model.predict(n=48, past_covariates=X_train))

        trial.report(intermediate_value, step)

        if trial.should_prune():
            raise optuna.TrialPruned()


    predictions = model.predict(n=48, past_covariates=X_train)
    return mae(y_test[:48], predictions)

In [ ]:
# Run Optuna optimization
pruner = optuna.pruners.MedianPruner()
sampler = optuna.samplers.TPESampler()

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=10)

[I 2024-12-17 10:11:54,400] A new study created in memory with name: no-name-67dca55c-8603-472a-8219-9801a74c4b96
<ipython-input-21-51f18f6fa1c5>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_D

Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/darts/metrics/metrics.py:979: RuntimeWarning: Mean of empty slice
  return np.nanmean(
/usr/local/lib/python3.10/dist-packages/darts/metrics/metrics.py:370: RuntimeWarning: Mean of empty slice
  vals = component_reduction(vals, axis=COMP_AX)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
# Print best parameters
print(study.best_value)
print(f"Best trial: {study.best_trial.params}")

## Alternatively Grid Search?

In [31]:
es = EarlyStopping(
  monitor="val_loss",
  patience=5,
  min_delta=0.05,
  mode='min',
)

trf_model = TransformerModel(
        input_chunk_length=48,
        output_chunk_length=48,
        batch_size=32,
        n_epochs=100,
        model_name="transformer_snowfall_grid",
        d_model=64,
        nhead=2,
        dim_feedforward=2,
        num_encoder_layers=2,
        num_decoder_layers=2,
        dropout=0.01,
        pl_trainer_kwargs={
            "accelerator": "auto",
            "callbacks": [es]},
        random_state=42
    )

param_grid = {
    "d_model": [128, 256, 512],
    "nhead": [4, 8],
    "num_encoder_layers": [2, 4, 6],
    "num_decoder_layers": [2, 4, 6],
    "dim_feedforward": [256, 512, 1024],
    "dropout": [0.1, 0.2],
    "activation": ["relu", "gelu"]
}

from itertools import product
import numpy as np

param_combinations = list(product(*param_grid.values()))

best_mae = float('inf')
best_params = {}

for params in param_combinations:
    d_model, nhead, num_encoder, num_decoder, dim_ff, dropout, activation = params
    model = TransformerModel(
        input_chunk_length=48,
        output_chunk_length=48,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=num_encoder,
        num_decoder_layers=num_decoder,
        dim_feedforward=dim_ff,
        dropout=dropout,
        activation=activation
    )

    trf_model.fit(
          series=y_train,
          past_covariates=X_train,
          val_series=y_val,
          val_past_covariates=X_val,
          verbose=False
        )

    forecast = trf_model.historical_forecasts(
        X_test,
        forecast_horizon=48,
        stride=1,
        retrain=False
    )

    current_mae = mae(y_test, forecast).mean().item()

    # Track the best RMSE
    if current_mae < best_mae:
        best_mae = mae
        best_params = {
            "d_model": d_model,
            "nhead": nhead,
            "num_encoder_layers": num_encoder,
            "num_decoder_layers": num_decoder,
            "dim_feedforward": dim_ff,
            "dropout": dropout,
            "activation": activation
        }

print(f"Best Hyperparameters: {best_params}")
print(f"Best MAE: {best_mae}")

/home/anita/.pyenv/versions/3.10.6/envs/powder_alert2.0/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
ValueError: Cannot build a single input for prediction with the provided model, `series` and `*_covariates` at series index: 0. The minimum prediction input time index requirements were not met. Please check the time index of `series` and `*_covariates`.


ValueError: Cannot build a single input for prediction with the provided model, `series` and `*_covariates` at series index: 0. The minimum prediction input time index requirements were not met. Please check the time index of `series` and `*_covariates`.

In [30]:
print(X_train.time_index)
print(X_val.time_index)
print(X_test.time_index)

DatetimeIndex(['2020-07-31 00:00:00', '2020-07-31 01:00:00',
               '2020-07-31 02:00:00', '2020-07-31 03:00:00',
               '2020-07-31 04:00:00', '2020-07-31 05:00:00',
               '2020-07-31 06:00:00', '2020-07-31 07:00:00',
               '2020-07-31 08:00:00', '2020-07-31 09:00:00',
               ...
               '2022-10-08 12:00:00', '2022-10-08 13:00:00',
               '2022-10-08 14:00:00', '2022-10-08 15:00:00',
               '2022-10-08 16:00:00', '2022-10-08 17:00:00',
               '2022-10-08 18:00:00', '2022-10-08 19:00:00',
               '2022-10-08 20:00:00', '2022-10-08 21:00:00'],
              dtype='datetime64[ns]', name='date', length=19198, freq='h')
DatetimeIndex(['2022-10-08 22:00:00', '2022-10-08 23:00:00',
               '2022-10-09 00:00:00', '2022-10-09 01:00:00',
               '2022-10-09 02:00:00', '2022-10-09 03:00:00',
               '2022-10-09 04:00:00', '2022-10-09 05:00:00',
               '2022-10-09 06:00:00', '2022-10-09 0

## Initialize Model with optimized Parameters

In [ ]:
#fill after completing optuna study!
model = TransformerModel(
    input_chunk_length=48,
    output_chunk_length=48,
    batch_size=32,
    n_epochs=20,
    model_name="snowfall_model",
    nr_epochs_val_period=1,
    d_model=64,
    nhead=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    dropout=0.1,
    random_state=42,
    likelihood=None  # Regression task
)


## Fit Model

In [ ]:
model.fit(
    series=y_train,
    past_covariates=X_train,
    val_series=y_val,
    val_past_covariates=X_val,
    verbose=True
)

## Make Predictions

In [ ]:
forecast = model.predict(n=48, past_covariates=X_test)